In [1]:
%autosave 0

Autosave disabled


In [2]:
import os
import pandas as pd
import numpy as np
import re
from ast import literal_eval
import collections
import pickle
import scipy

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Get the current directory
current_dir = os.getcwd()
current_dir

'/content'

#Explore

In [4]:
dir_warmze = '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/AOP_DP_Analytics/Data/Instances/Warmsen'

# Read the CSV file using the relative file path
po_warmze = pd.read_csv(dir_warmze + '/post_order_id_mapping.dat', sep='\t', names=('PostObjectId', 'RoutePosID'))
pp_warmze = pd.read_csv(dir_warmze + '/post_point_information.dat', sep='\t', names=('PostPointId', 'PostObjectId'))

In [ ]:
po_warmze.head()

,PostObjectId,RoutePosID
0,16,AD14E08623FF9F5AD2293E7DEDB4F4B1
1,17,F48CB0CAB0BC117AE030007F0100574C
2,18,F48CB0CAB0C7117AE030007F0100574C
3,19,F48CB0CAB0C8117AE030007F0100574C
4,20,069AE286923C1800E040400A09131416


In [ ]:
pp_warmze.head()

,PostPointId,PostObjectId
0,0,"[0, 113]"
1,1,[1]
2,2,[2]
3,3,[3]
4,4,[4]


In [ ]:
pp_warmze.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5226 entries, 0 to 5225
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PostPointId   5226 non-null   int64 
 1   PostObjectId  5226 non-null   object
dtypes: int64(1), object(1)
memory usage: 81.8+ KB


In [ ]:
# Convert 'PostObjectId' column from string to list
pp_warmze['PostObjectId'] = pp_warmze['PostObjectId'].apply(literal_eval)

# Use explode() to split the "PostObjectId" column into different rows
pp_warmze = pp_warmze.explode('PostObjectId', ignore_index=True)
pp_warmze.head(10)

,PostPointId,PostObjectId
0,0,0
1,0,113
2,1,1
3,2,2
4,3,3
5,4,4
6,5,5
7,6,6
8,7,7
9,8,8


#Districts

In [5]:
district_path = '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/AOP_DP_Analytics/Data/Instances/Warmsen/Districts'

# for filename in os.scandir(district_path):
for filename in os.listdir(district_path):
  print(filename)
# for filename in os.walk(district_path):
    # if filename.is_file():
    #   print(filename)
        # print(filename.path)

31603-06.dat
31604-10.dat
31600-01.dat
31603-08.dat
31606-12.dat
31606-11.dat
31600-02.dat
31600-03.dat
31600-04.dat
31603-05.dat
31603-07.dat
31606-13.dat
31604-09.dat
31606-14.dat


In [6]:
district_path = '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/AOP_DP_Analytics/Data/Instances/Warmsen/Districts'

for filename in os.listdir(district_path):
  file_var = re.sub('.dat', '_warmze', filename)
  print(file_var)

31603-06_warmze
31604-10_warmze
31600-01_warmze
31603-08_warmze
31606-12_warmze
31606-11_warmze
31600-02_warmze
31600-03_warmze
31600-04_warmze
31603-05_warmze
31603-07_warmze
31606-13_warmze
31604-09_warmze
31606-14_warmze


In [7]:
%%time

district_path = '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/AOP_DP_Analytics/Data/Instances/Warmsen/Districts'

warmze_districts_list = []

for filename in os.listdir(district_path):
  file_var = re.sub('.dat', '_warmze', filename)
  file_district = pd.read_csv(district_path + '/' + filename, sep='\t', skiprows = [0,1],
                              names=('PostObjectID', 'dum_1', 'dum_2', 'dum_3', 'dum_4', 'dum_5')
                              # usecols = [0]
                              )
  file_district['district'] = file_var
  warmze_districts_list.append(file_district)

warmze_districts_df = pd.concat(warmze_districts_list, ignore_index = True)
warmze_districts_df.drop(['dum_1', 'dum_2', 'dum_3', 'dum_4', 'dum_5'], axis = 1, inplace = True)
warmze_districts_df['region'] = 'Warmze'

CPU times: user 71.2 ms, sys: 6.09 ms, total: 77.3 ms
Wall time: 3.75 s


In [ ]:
warmze_districts_list[0].head()

,PostObjectID,dum_1,dum_2,dum_3,dum_4,dum_5,district
0,2350,0,1,0,0,0,31603-06_warmze
1,2349,0,0,0,0,0,31603-06_warmze
2,2348,0,3,0,0,0,31603-06_warmze
3,2346,0,2,0,0,0,31603-06_warmze
4,2347,0,1,0,0,0,31603-06_warmze


In [ ]:
warmze_districts_df.head()

,PostObjectID,district,region
0,2350,31603-06_warmze,Warmze
1,2349,31603-06_warmze,Warmze
2,2348,31603-06_warmze,Warmze
3,2346,31603-06_warmze,Warmze
4,2347,31603-06_warmze,Warmze


In [ ]:
warmze_districts_df['district'].value_counts()

31603-06_warmze    445
31600-01_warmze    440
31606-12_warmze    435
31600-02_warmze    434
31604-09_warmze    434
31603-07_warmze    423
31604-10_warmze    412
31600-03_warmze    394
31603-05_warmze    393
31603-08_warmze    386
31606-11_warmze    380
31606-14_warmze    338
31606-13_warmze    325
31600-04_warmze    300
Name: district, dtype: int64

In [ ]:
warmze_districts_df['PostObjectID'].value_counts()

2350    1
1996    1
2240    1
2241    1
2008    1
       ..
5431    1
5429    1
5430    1
5426    1
32      1
Name: city_id, Length: 5539, dtype: int64

#Post Order ID Mapping

In [ ]:
po_warmze.head()

,PostObjectId,RoutePosID
0,16,AD14E08623FF9F5AD2293E7DEDB4F4B1
1,17,F48CB0CAB0BC117AE030007F0100574C
2,18,F48CB0CAB0C7117AE030007F0100574C
3,19,F48CB0CAB0C8117AE030007F0100574C
4,20,069AE286923C1800E040400A09131416


In [ ]:
po_warmze.shape

(5539, 2)

In [ ]:
po_warmze['PostObjectId'].value_counts()

16      1
3705    1
3713    1
3712    1
3711    1
       ..
1861    1
1860    1
1859    1
1858    1
5554    1
Name: PostObjectId, Length: 5539, dtype: int64

In [ ]:
po_warmze['RoutePosID'].value_counts()

AD14E08623FF9F5AD2293E7DEDB4F4B1    1
F48CB0CABB33117AE030007F0100574C    1
11DE6DAA72799EA418204C74A59010C7    1
F48CB0CABB3A117AE030007F0100574C    1
F48CB0CABB39117AE030007F0100574C    1
                                   ..
507C1274AA243617E040400A09137C3A    1
507C1274AA233617E040400A09137C3A    1
507C1274AA223617E040400A09137C3A    1
F48CB0CAB7CA117AE030007F0100574C    1
F48CB0CADA3C117AE030007F0100574C    1
Name: RoutePosID, Length: 5539, dtype: int64

In [8]:
# po_warmze_dict = po_warmze.to_dict(orient = 'list')
po_warmze_dict = dict(zip(po_warmze.PostObjectId, po_warmze.RoutePosID))

In [ ]:
po_warmze_dict

{16: 'AD14E08623FF9F5AD2293E7DEDB4F4B1',
 17: 'F48CB0CAB0BC117AE030007F0100574C',
 18: 'F48CB0CAB0C7117AE030007F0100574C',
 19: 'F48CB0CAB0C8117AE030007F0100574C',
 20: '069AE286923C1800E040400A09131416',
 21: 'F48CB0CAB0BF117AE030007F0100574C',
 22: 'F48CB0CAB0C0117AE030007F0100574C',
 23: 'F48CB0CAB0C1117AE030007F0100574C',
 24: 'F48CB0CAB0C3117AE030007F0100574C',
 25: 'F48CB0CAB0C4117AE030007F0100574C',
 26: 'F48CB0CACE02117AE030007F0100574C',
 27: 'F48CB0CACE03117AE030007F0100574C',
 28: 'F48CB0CACE04117AE030007F0100574C',
 29: 'F48CB0CACE05117AE030007F0100574C',
 30: 'F48CB0CACE06117AE030007F0100574C',
 31: 'F48CB0CAD9A0117AE030007F0100574C',
 32: 'F48CB0CAD9A1117AE030007F0100574C',
 33: '3A3F3791F26C5CD616DD258B78070959',
 34: '6AE0342C41FAF128E040400A0913533A',
 35: 'F48CB0CAB0B7117AE030007F0100574C',
 36: 'F48CB0CAB0B8117AE030007F0100574C',
 37: 'F48CB0CAB0BA117AE030007F0100574C',
 38: 'F48CB0CAD9DF117AE030007F0100574C',
 39: 'F48CB0CAD9E0117AE030007F0100574C',
 40: 'F48CB0CAD9

In [9]:
#Update Warmzen dataframe

%%time

def PosObjectMapping(po):
  try:
    return po_warmze_dict[po]
  except:
    return 'null'

warmze_districts_df['RoutePosID'] = warmze_districts_df['PostObjectID'].apply(lambda x : PosObjectMapping(x))

CPU times: user 5.24 ms, sys: 602 µs, total: 5.84 ms
Wall time: 6.91 ms


In [ ]:
warmze_districts_df.head()

,PostObjectID,district,region,RoutePosID
0,2350,31603-06_warmze,Warmze,F48CB0CACD49117AE030007F0100574C
1,2349,31603-06_warmze,Warmze,F48CB0CACD48117AE030007F0100574C
2,2348,31603-06_warmze,Warmze,F48CB0CACD47117AE030007F0100574C
3,2346,31603-06_warmze,Warmze,F48CB0CACD44117AE030007F0100574C
4,2347,31603-06_warmze,Warmze,F48CB0CACD45117AE030007F0100574C


In [ ]:
warmze_districts_df['RoutePosID'].value_counts()

F48CB0CACD49117AE030007F0100574C    1
C6538040EF4494AB1E7B14BADC9C8A98    1
5AEE015051382A2F6D3D0B67E028442F    1
F48CB0CAC14C117AE030007F0100574C    1
F48CB0CAC026117AE030007F0100574C    1
                                   ..
F48CB0CAB1DF117AE030007F0100574C    1
F48CB0CAB1DD117AE030007F0100574C    1
F48CB0CAB1DE117AE030007F0100574C    1
F48CB0CAB1D9117AE030007F0100574C    1
F48CB0CAD9A1117AE030007F0100574C    1
Name: RoutePosID, Length: 5539, dtype: int64

#Weekday Volume

In [10]:
volume_path = '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/AOP_DP_Analytics/Data/Volumes'

tes = pd.read_csv(volume_path + '/routepos_vm_mo.csv', sep=';')
tes.rename(columns = {'ROUTEPOS_ID': 'RoutePosID'}, inplace = True)

In [ ]:
tes.head()

,RoutePosID,BRIEFE,PAKETE,SONSTIGE
0,61ADE0B658BD7433E040400A0913386C,0.1510,0.1185,0.0
1,6F994C12508952C82A3BD63E30AA0606,0.1510,0.1185,0.0
2,D6590A6EA73C629B096242214AB6FA85,0.1510,0.1185,0.0
3,00F6207EED20EAE0E040007F01000C8B,0.0625,0.0725,0.0
4,30B7851A725D50BBE040400A09130368,0.0625,0.0725,0.0


In [ ]:
tes.shape

(754563, 4)

In [11]:
warmze_districts_df.shape

(5539, 4)

In [12]:
tes_2 = pd.merge(warmze_districts_df, tes, on='RoutePosID', how='left')
tes_2.shape

(5539, 7)

In [ ]:
tes_2.head()

,PostObjectID,district,region,RoutePosID,BRIEFE,PAKETE,SONSTIGE
0,2350,31603-06_warmze,Warmze,F48CB0CACD49117AE030007F0100574C,0.051053,0.026316,0.0
1,2349,31603-06_warmze,Warmze,F48CB0CACD48117AE030007F0100574C,0.000000,0.000000,0.0
2,2348,31603-06_warmze,Warmze,F48CB0CACD47117AE030007F0100574C,0.153158,0.078947,0.0
3,2346,31603-06_warmze,Warmze,F48CB0CACD44117AE030007F0100574C,0.102105,0.052632,0.0
4,2347,31603-06_warmze,Warmze,F48CB0CACD45117AE030007F0100574C,0.051053,0.026316,0.0


In [13]:
for filename in os.listdir(volume_path):
  print(filename)

routepos_vm_di.csv
routepos_vm_do.csv
routepos_vm_fr.csv
routepos_vm_mi.csv
routepos_vm_mo.csv
routepos_vm_sa.csv


In [15]:
dot = warmze_districts_df.copy()

In [16]:
%%time

file_day_mapping = {
    'routepos_vm_mo.csv' : 'Montag',
    'routepos_vm_di.csv' : 'Dienstag',
    'routepos_vm_mi.csv' : 'Mittwoch',
    'routepos_vm_do.csv' : 'Donnerstag',
    'routepos_vm_fr.csv' : 'Freitag',
    'routepos_vm_sa.csv' : 'Samstag'
}

volume_path = '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/AOP_DP_Analytics/Data/Volumes'

warmze_volume_list = []

dot['BRIEFE'] = ''
dot['PAKETE'] = ''
dot['SONSTIGE'] = ''

for filename in os.listdir(volume_path):
  file_volume = pd.read_csv(volume_path + '/' + filename, sep=';')
  file_volume['day'] = file_day_mapping[filename]
  file_volume.rename(columns = {'ROUTEPOS_ID': 'RoutePosID'}, inplace = True)

  dot = warmze_districts_df.copy()
  dot = pd.merge(dot, file_volume, on='RoutePosID', how='left')

  warmze_volume_list.append(dot)

dot_all = pd.concat(warmze_volume_list, ignore_index = True)

CPU times: user 10.8 s, sys: 823 ms, total: 11.6 s
Wall time: 16.2 s


In [ ]:
dot.head()

,PostObjectID,district,region,RoutePosID,BRIEFE,PAKETE,SONSTIGE,day
0,2350,31603-06_warmze,Warmze,F48CB0CACD49117AE030007F0100574C,0.592632,0.091053,0.492105,Samstag
1,2349,31603-06_warmze,Warmze,F48CB0CACD48117AE030007F0100574C,0.000000,0.000000,0.000000,Samstag
2,2348,31603-06_warmze,Warmze,F48CB0CACD47117AE030007F0100574C,1.777895,0.273158,1.476316,Samstag
3,2346,31603-06_warmze,Warmze,F48CB0CACD44117AE030007F0100574C,1.185263,0.182105,0.984211,Samstag
4,2347,31603-06_warmze,Warmze,F48CB0CACD45117AE030007F0100574C,0.592632,0.091053,0.492105,Samstag


In [ ]:
dot_all.head()

,PostObjectID,district,region,RoutePosID,BRIEFE,PAKETE,SONSTIGE,day
0,2350,31603-06_warmze,Warmze,F48CB0CACD49117AE030007F0100574C,0.477895,0.098421,0.316316,Dienstag
1,2349,31603-06_warmze,Warmze,F48CB0CACD48117AE030007F0100574C,0.000000,0.000000,0.000000,Dienstag
2,2348,31603-06_warmze,Warmze,F48CB0CACD47117AE030007F0100574C,1.433684,0.295263,0.948947,Dienstag
3,2346,31603-06_warmze,Warmze,F48CB0CACD44117AE030007F0100574C,0.955789,0.196842,0.632632,Dienstag
4,2347,31603-06_warmze,Warmze,F48CB0CACD45117AE030007F0100574C,0.477895,0.098421,0.316316,Dienstag


In [ ]:
dot_all['day'].value_counts()

Dienstag      5539
Donnerstag    5539
Freitag       5539
Mittwoch      5539
Montag        5539
Samstag       5539
Name: day, dtype: int64

In [ ]:
dot_all['district'].value_counts()

31603-06_warmze    2670
31600-01_warmze    2640
31606-12_warmze    2610
31600-02_warmze    2604
31604-09_warmze    2604
31603-07_warmze    2538
31604-10_warmze    2472
31600-03_warmze    2364
31603-05_warmze    2358
31603-08_warmze    2316
31606-11_warmze    2280
31606-14_warmze    2028
31606-13_warmze    1950
31600-04_warmze    1800
Name: district, dtype: int64

In [ ]:
warmze_districts_df['district'].value_counts()*6

31603-06_warmze    2670
31600-01_warmze    2640
31606-12_warmze    2610
31600-02_warmze    2604
31604-09_warmze    2604
31603-07_warmze    2538
31604-10_warmze    2472
31600-03_warmze    2364
31603-05_warmze    2358
31603-08_warmze    2316
31606-11_warmze    2280
31606-14_warmze    2028
31606-13_warmze    1950
31600-04_warmze    1800
Name: district, dtype: int64

#Instance Creation Using Poisson 

In [ ]:
rng = np.random.default_rng()
s = rng.poisson(lam = 1.5, size = 20)

In [ ]:
s

array([1, 0, 1, 2, 2, 4, 2, 3, 1, 0, 2, 1, 3, 0, 0, 0, 1, 2, 1, 1])

In [ ]:
counter = collections.Counter(s)
counter

Counter({1: 7, 0: 5, 2: 5, 4: 1, 3: 2})

In [ ]:
scipy.stats.mode(s)

<ipython-input-118-6da934112331>:1: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  scipy.stats.mode(s)


ModeResult(mode=array([1]), count=array([7]))

In [ ]:
scipy.stats.mode(s, keepdims = True)[0][0]

1

In [17]:
# scenario 1 returns mode from poisson distr
# idea is the instance volumes will be floor or upper of given expected volume

def scenario_1(pos_delivery):
  rng = np.random.default_rng()
  binom_dist = rng.poisson(lam = pos_delivery, size = 52)

  return scipy.stats.mode(binom_dist, keepdims = True)[0][0]

In [ ]:
warmze_grouped = dot_all.groupby(by = ['district', 'day'])
len(warmze_grouped)

84

In [ ]:
for name, group in warmze_grouped:
  print(name)

('31600-01_warmze', 'Dienstag')
('31600-01_warmze', 'Donnerstag')
('31600-01_warmze', 'Freitag')
('31600-01_warmze', 'Mittwoch')
('31600-01_warmze', 'Montag')
('31600-01_warmze', 'Samstag')
('31600-02_warmze', 'Dienstag')
('31600-02_warmze', 'Donnerstag')
('31600-02_warmze', 'Freitag')
('31600-02_warmze', 'Mittwoch')
('31600-02_warmze', 'Montag')
('31600-02_warmze', 'Samstag')
('31600-03_warmze', 'Dienstag')
('31600-03_warmze', 'Donnerstag')
('31600-03_warmze', 'Freitag')
('31600-03_warmze', 'Mittwoch')
('31600-03_warmze', 'Montag')
('31600-03_warmze', 'Samstag')
('31600-04_warmze', 'Dienstag')
('31600-04_warmze', 'Donnerstag')
('31600-04_warmze', 'Freitag')
('31600-04_warmze', 'Mittwoch')
('31600-04_warmze', 'Montag')
('31600-04_warmze', 'Samstag')
('31603-05_warmze', 'Dienstag')
('31603-05_warmze', 'Donnerstag')
('31603-05_warmze', 'Freitag')
('31603-05_warmze', 'Mittwoch')
('31603-05_warmze', 'Montag')
('31603-05_warmze', 'Samstag')
('31603-06_warmze', 'Dienstag')
('31603-06_warmze'

In [ ]:
# %%time

# for name, group in warmze_grouped:
#   current_group = warmze_grouped.get_group(name)

#   current_group['all_deliv'] = current_group['BRIEFE'] + current_group['PAKETE'] + current_group['SONSTIGE']

#   current_group['scenario_1_all'] = current_group['all_deliv'].apply(lambda x :scenario_1(x))
#   current_group['scenario_1_briefe'] = current_group['BRIEFE'].apply(lambda x :scenario_1(x))
#   current_group['scenario_1_pakete'] = current_group['PAKETE'].apply(lambda x :scenario_1(x))
#   # current_group = current_group.assign()

In [ ]:
warmze_grouped.get_group(('31604-10_warmze', 'Mittwoch')).head()

,PostObjectID,district,region,RoutePosID,BRIEFE,PAKETE,SONSTIGE,day
17062,4067,31604-10_warmze,Warmze,F48CB0CACDAA117AE030007F0100574C,1.003889,0.160556,0.304444,Mittwoch
17063,4053,31604-10_warmze,Warmze,ACCE5E829CDD20E9E0403AA03E585E92,0.000000,0.000000,0.000000,Mittwoch
17064,4066,31604-10_warmze,Warmze,F48CB0CACDA9117AE030007F0100574C,1.003889,0.160556,0.304444,Mittwoch
17065,4052,31604-10_warmze,Warmze,A28624836800EEA8B5AAC65AEAD0179C,1.003889,0.160556,0.304444,Mittwoch
17066,4065,31604-10_warmze,Warmze,F48CB0CACDA6117AE030007F0100574C,1.003889,0.160556,0.304444,Mittwoch


In [ ]:
# %%time

# dot_all['all_deliv'] = dot_all['BRIEFE'] + dot_all['PAKETE'] + dot_all['SONSTIGE']

# dot_all['scenario_1_all'] = dot_all['all_deliv'].apply(lambda x :scenario_1(x))
# dot_all['scenario_1_briefe'] = dot_all['BRIEFE'].apply(lambda x :scenario_1(x))
# dot_all['scenario_1_pakete'] = dot_all['PAKETE'].apply(lambda x :scenario_1(x))

CPU times: user 16.6 s, sys: 174 ms, total: 16.7 s
Wall time: 16.9 s


In [20]:
def generate_scenarios(df, scenario_type, scenario_number):
  sce_briefe = 'scenario_' + str(scenario_number) + '_briefe'
  sce_pakete = 'scenario_' + str(scenario_number) + '_pakete'
  dum_sonstige = 'sonstige_dum'

  df[sce_briefe] = df['BRIEFE'].apply(lambda x :scenario_type(x))
  df[sce_pakete] = df['PAKETE'].apply(lambda x :scenario_type(x))
  df[dum_sonstige] = df['SONSTIGE'].apply(lambda x :scenario_type(x))

  sce_all = 'scenario_' + str(scenario_number) + '_all'
  df[sce_all] = df[sce_briefe] + df[sce_pakete] + df[dum_sonstige]

  df.drop(columns = [dum_sonstige], axis = 1, inplace = True)

  return df

In [22]:
%%time

dot_all = generate_scenarios(df = dot_all, scenario_type = scenario_1, scenario_number = 1)

CPU times: user 16.4 s, sys: 203 ms, total: 16.6 s
Wall time: 16.9 s


In [23]:
dot_all.head()

,PostObjectID,district,region,RoutePosID,BRIEFE,PAKETE,SONSTIGE,day,scenario_1_briefe,scenario_1_pakete,scenario_1_all
0,2350,31603-06_warmze,Warmze,F48CB0CACD49117AE030007F0100574C,0.477895,0.098421,0.316316,Dienstag,0,0,0
1,2349,31603-06_warmze,Warmze,F48CB0CACD48117AE030007F0100574C,0.000000,0.000000,0.000000,Dienstag,0,0,0
2,2348,31603-06_warmze,Warmze,F48CB0CACD47117AE030007F0100574C,1.433684,0.295263,0.948947,Dienstag,1,0,1
3,2346,31603-06_warmze,Warmze,F48CB0CACD44117AE030007F0100574C,0.955789,0.196842,0.632632,Dienstag,1,0,1
4,2347,31603-06_warmze,Warmze,F48CB0CACD45117AE030007F0100574C,0.477895,0.098421,0.316316,Dienstag,0,0,0


In [24]:
dot_all_grouped = dot_all.groupby(by = ['district', 'day'])

In [25]:
dot_all_grouped.get_group(('31604-10_warmze', 'Mittwoch')).shape

(412, 11)

In [26]:
warmze_districts_df['district'].value_counts()

31603-06_warmze    445
31600-01_warmze    440
31606-12_warmze    435
31600-02_warmze    434
31604-09_warmze    434
31603-07_warmze    423
31604-10_warmze    412
31600-03_warmze    394
31603-05_warmze    393
31603-08_warmze    386
31606-11_warmze    380
31606-14_warmze    338
31606-13_warmze    325
31600-04_warmze    300
Name: district, dtype: int64

In [27]:
dot_all_grouped.get_group(('31604-10_warmze', 'Mittwoch'))

,PostObjectID,district,region,RoutePosID,BRIEFE,PAKETE,SONSTIGE,day,scenario_1_briefe,scenario_1_pakete,scenario_1_all
17062,4067,31604-10_warmze,Warmze,F48CB0CACDAA117AE030007F0100574C,1.003889,0.160556,0.304444,Mittwoch,1,0,1
17063,4053,31604-10_warmze,Warmze,ACCE5E829CDD20E9E0403AA03E585E92,0.000000,0.000000,0.000000,Mittwoch,0,0,0
17064,4066,31604-10_warmze,Warmze,F48CB0CACDA9117AE030007F0100574C,1.003889,0.160556,0.304444,Mittwoch,0,0,0
17065,4052,31604-10_warmze,Warmze,A28624836800EEA8B5AAC65AEAD0179C,1.003889,0.160556,0.304444,Mittwoch,1,0,1
17066,4065,31604-10_warmze,Warmze,F48CB0CACDA6117AE030007F0100574C,1.003889,0.160556,0.304444,Mittwoch,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
17469,4234,31604-10_warmze,Warmze,F48CB0CACBE5117AE030007F0100574C,1.007692,0.133846,0.233077,Mittwoch,1,0,1
17470,4227,31604-10_warmze,Warmze,5077EB77D3E2E5A3E040400A09133CDE,1.007692,0.133846,0.233077,Mittwoch,0,0,0
17471,4235,31604-10_warmze,Warmze,F48CB0CACBE7117AE030007F0100574C,2.015385,0.267692,0.466154,Mittwoch,1,0,1
17472,4236,31604-10_warmze,Warmze,F48CB0CACBE8117AE030007F0100574C,1.007692,0.133846,0.233077,Mittwoch,0,0,0


In [30]:
dot_all['scenario_1_pakete'].value_counts(normalize = True)

0    0.996720
1    0.002166
6    0.000271
5    0.000271
2    0.000211
8    0.000090
7    0.000090
4    0.000090
3    0.000060
9    0.000030
Name: scenario_1_pakete, dtype: float64

In [32]:
dot_all_grouped.get_group(('31604-10_warmze', 'Mittwoch'))['scenario_1_pakete'].value_counts()

0    411
1      1
Name: scenario_1_pakete, dtype: int64

In [38]:
dot_all_grouped.get_group(('31604-10_warmze', 'Mittwoch'))['scenario_1_briefe'].value_counts()

0    184
1    181
2     38
3      9
Name: scenario_1_briefe, dtype: int64

In [39]:
dot_all_grouped.get_group(('31604-10_warmze', 'Mittwoch'))['scenario_1_all'].value_counts()

0    184
1    181
2     36
3     11
Name: scenario_1_all, dtype: int64

In [33]:
dot_all_grouped.get_group(('31604-10_warmze', 'Mittwoch'))

,PostObjectID,district,region,RoutePosID,BRIEFE,PAKETE,SONSTIGE,day,scenario_1_briefe,scenario_1_pakete,scenario_1_all
17062,4067,31604-10_warmze,Warmze,F48CB0CACDAA117AE030007F0100574C,1.003889,0.160556,0.304444,Mittwoch,1,0,1
17063,4053,31604-10_warmze,Warmze,ACCE5E829CDD20E9E0403AA03E585E92,0.000000,0.000000,0.000000,Mittwoch,0,0,0
17064,4066,31604-10_warmze,Warmze,F48CB0CACDA9117AE030007F0100574C,1.003889,0.160556,0.304444,Mittwoch,0,0,0
17065,4052,31604-10_warmze,Warmze,A28624836800EEA8B5AAC65AEAD0179C,1.003889,0.160556,0.304444,Mittwoch,1,0,1
17066,4065,31604-10_warmze,Warmze,F48CB0CACDA6117AE030007F0100574C,1.003889,0.160556,0.304444,Mittwoch,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
17469,4234,31604-10_warmze,Warmze,F48CB0CACBE5117AE030007F0100574C,1.007692,0.133846,0.233077,Mittwoch,1,0,1
17470,4227,31604-10_warmze,Warmze,5077EB77D3E2E5A3E040400A09133CDE,1.007692,0.133846,0.233077,Mittwoch,0,0,0
17471,4235,31604-10_warmze,Warmze,F48CB0CACBE7117AE030007F0100574C,2.015385,0.267692,0.466154,Mittwoch,1,0,1
17472,4236,31604-10_warmze,Warmze,F48CB0CACBE8117AE030007F0100574C,1.007692,0.133846,0.233077,Mittwoch,0,0,0


In [35]:
dot_all_grouped.get_group(('31604-10_warmze', 'Mittwoch'))['PAKETE'].quantile(q = 0.5)

0.1771428571428571

#Save Instances

In [40]:
save_loc = '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding'

pickle.dump(dot_all_grouped, open(save_loc + "/warmze_instance_scenario_1.p", "wb"))

# we have instances for all districts in warmze everyday
# instances for briefe, pakete, and all
# we have 84 * 3 = 252 instances

In [37]:
warmze_instance_scenario_1 = pickle.load(open(save_loc + "/warmze_instance_scenario_1.p", "rb"))